In [1]:
import pandas as pd
import numpy as np

In [2]:
import edhec_risk_kit_204 as erk

/Users/Lucas_Hattori/Projects/coursera-AdvancedPortfolioConstruction/original_notebooks/edhec_risk_kit_204.py:43: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if filetype is "returns":
/Users/Lucas_Hattori/Projects/coursera-AdvancedPortfolioConstruction/original_notebooks/edhec_risk_kit_204.py:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif filetype is "nfirms":
/Users/Lucas_Hattori/Projects/coursera-AdvancedPortfolioConstruction/original_notebooks/edhec_risk_kit_204.py:49: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif filetype is "size":


# Load data

In [7]:
industries_to_analysis = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food',]

In [8]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"]
ind49_rets = ind49_rets[industries_to_analysis]
ind49_rets

,Hlth,Fin,Whlsl,Rtail,Food
2013-01,0.0966,0.1300,0.0646,0.0541,0.0754
2013-02,0.0078,0.0177,0.0207,0.0061,0.0483
2013-03,0.0591,0.0360,0.0325,0.0477,0.0798
2013-04,-0.0034,-0.0002,0.0017,0.0320,0.0174
2013-05,0.0454,0.0842,0.0385,0.0239,-0.0201
...,...,...,...,...,...
2018-08,0.0585,0.0004,0.0366,0.0911,0.0114
2018-09,-0.0045,-0.0340,-0.0148,0.0061,-0.0220
2018-10,-0.0659,-0.0455,-0.0788,-0.1021,0.0095
2018-11,0.0356,0.0006,0.0318,0.0159,0.0029


In [9]:
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2013":"2018"]
ind_mcap = ind_mcap[industries_to_analysis]
ind_mcap

,Hlth,Fin,Whlsl,Rtail,Food
2013-01,0.005221,0.021975,0.012207,0.068470,0.017441
2013-02,0.005426,0.023490,0.012226,0.068400,0.017752
2013-03,0.005402,0.023326,0.012332,0.068019,0.018379
2013-04,0.005509,0.023209,0.012210,0.068507,0.019086
2013-05,0.005029,0.022887,0.012057,0.069722,0.019140
...,...,...,...,...,...
2018-08,0.005456,0.024948,0.010987,0.076225,0.013758
2018-09,0.005577,0.024080,0.010992,0.080234,0.013394
2018-10,0.005383,0.023262,0.010681,0.080886,0.012858
2018-11,0.005370,0.024093,0.010674,0.078849,0.014076


# Black-Litterman

In [10]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)


In [32]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w


In [11]:
def sample_cov(r, **kwargs):
    """
    Returns the sample covariance of the supplied returns
    """
    return r.cov()

In [13]:
cov_matrix = sample_cov(ind49_rets) * np.sqrt(12)
cov_matrix

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.007651,0.004138,0.004149,0.003812,0.002191
Fin,0.004138,0.008124,0.005108,0.004072,0.002083
Whlsl,0.004149,0.005108,0.005311,0.004113,0.002658
Rtail,0.003812,0.004072,0.004113,0.005759,0.002940
Food,0.002191,0.002083,0.002658,0.002940,0.004540


In [25]:
cap_weigth = (ind_mcap.iloc[0] / ind_mcap.iloc[0].sum())
cap_weigth.sort_values()

Hlth     0.041663
Whlsl    0.097411
Food     0.139176
Fin      0.175362
Rtail    0.546388
Name: 2013-01, dtype: float64

In [34]:
pi = implied_returns(delta=2.5, sigma=cov_matrix, w=cap_weigth).sort_values()
pi.to_frame().style.format("{:.3%}")

,Implied Returns
Food,0.738%
Hlth,0.959%
Whlsl,1.051%
Fin,1.152%
Rtail,1.207%


## Relative views

In [47]:
# Germany will outperform other European Equities (i.e. FR and UK) by 5%
q = pd.Series([.05]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(industries_to_analysis), index=industries_to_analysis).T
# find the relative market caps of FR and UK to split the
# relative outperformance of DE ...
w_rtail = cap_weigth.loc["Rtail"]/(cap_weigth.loc["Rtail"]+cap_weigth.loc["Whlsl"])
w_whlsl = cap_weigth.loc["Whlsl"]/(cap_weigth.loc["Rtail"]+cap_weigth.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_rtail
p.iloc[0]['Whlsl'] = -w_whlsl
(p*100).round(2)

,Hlth,Fin,Whlsl,Rtail,Food
0,100.0,0.0,-15.13,-84.87,0.0


In [48]:
delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(cap_weigth, cov_matrix, p, q, tau = tau)
(bl_mu*100).sort_values().round(1)


Rtail    0.4
Food     0.4
Whlsl    1.0
Fin      1.1
Hlth     2.8
dtype: float64

In [49]:
# Use the Black Litterman expected returns and covariance matrix
w_msr(bl_sigma, bl_mu)

Hlth     2.075768
Fin      0.175362
Whlsl   -0.210362
Rtail   -1.179944
Food     0.139176
dtype: float64